# LLM Session Memory

Large Language Models are inherently stateless and have no knowledge of previous interactions with a user, or even of previous parts of the current conversation. While this may not be noticable when asking simple questions, it becomes a hinderance when engaging in long running conversations that rely on conversational context.

The solution to this problem is to append the previous conversation history to each subsequent call to the LLM.

This notebook will show how to use Redis to structure and store and retrieve this conversational session memory.

In [1]:
from redisvl.extensions.session_manager import StandardSessionManager
chat_session = StandardSessionManager(name='student tutor')

11:19:30 redisvl.index.index INFO   Index already exists, not overwriting.


To align with common LLM APIs, Redis stores messages with `role` and `content` fields.
The supported roles are "system", "user" and "llm".

You can store messages one at a time or all at once.

In [2]:
chat_session.add_message({"role":"system", "content":"You are a helpful geography tutor, giving simple and short answers to questions about Europen countries."})
chat_session.add_messages([
                            {"role":"user", "content":"What is the capital of France?"},
                            {"role":"llm", "content":"The capital is Paris."},
                            {"role":"user", "content":"And what is the capital of Spain?"},
                            {"role":"llm", "content":"The capital is Madrid."},
                            {"role":"user", "content":"What is the population of Great Britain?"},
                            {"role":"llm", "content":"As of 2023 the population of Great Britain is approximately 67 million people."},
                            ]
                          )

11:19:30 redisvl.index.index ERROR   Error while loading data to Redis
Traceback (most recent call last):
  File "/Users/justin.cechmanek/Documents/redisvl/redisvl/index/storage.py", line 70, in _create_key
    key_value = obj[id_field]  # type: ignore
                ~~~^^^^^^^^^^
KeyError: '_id'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/justin.cechmanek/Documents/redisvl/redisvl/index/index.py", line 587, in load
    return self._storage.write(
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/justin.cechmanek/Documents/redisvl/redisvl/index/storage.py", line 220, in write
    else self._create_key(obj, id_field)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/justin.cechmanek/Documents/redisvl/redisvl/index/storage.py", line 72, in _create_key
    raise ValueError(f"Key field {id_field} not found in record {obj}")
ValueError: Key field _id not found in record {'entry_id': None, 'role': 'system', 'cont

ValueError: Key field _id not found in record {'entry_id': None, 'role': 'system', 'content': 'You are a helpful geography tutor, giving simple and short answers to questions about Europen countries.', 'session_tag': 'c9cc1d8d-c7cb-4e33-887e-87f04594d44a', 'timestamp': 1722536370.422074}

At any point we can retrieve the recent history of the conversation. It will be ordered by entry time.

In [ ]:
context = chat_session.get_recent()
for message in context:
    print(message)

In many LLM flows the conversation progresses in a series of prompt and response pairs. session managers offer a convienience function `store()` to add these simply.

In [ ]:
prompt = "what is the size of England compared to Portugal?"
response = "England is larger in land area than Portal by about 15000 square miles."
chat_session.store(prompt, response)

context = chat_session.get_recent(top_k=6)
for message in context:
    print(message)

## Managing multiple users and conversations

For applications that need to handle multiple conversations concurrently, Redis supports tagging messages to keep conversations separated.

In [ ]:
chat_session.add_message({"role":"system", "content":"You are a helpful algebra tutor, giving simple answers to math problems."}, session_tag='student two')
chat_session.add_messages([
                            {"role":"user", "content":"What is the value of x in the equation 2x + 3 = 7?"},
                            {"role":"llm", "content":"The value of x is 2."},
                            {"role":"user", "content":"What is the value of y in the equation 3y - 5 = 7?"},
                            {"role":"llm", "content":"The value of y is 4."}
                            ], session_tag='student two')

for math_message in chat_session.get_recent(session_tag='student two'):
    print(math_message)

## Semantic conversation memory
For longer conversations our list of messages keeps growing. Since LLMs are stateless we have to continue to pass this conversation history on each subsequent call to ensure the LLM has the correct context.

A typical flow looks like this:
```
while True:
    prompt = input('enter your next question')
    context = chat_session.get_recent()
    response = LLM_api_call(prompt=prompt, context=context)
    chat_session.store(prompt, response)
```

This works, but as context keeps growing so too does our LLM token count, which increases latency and cost.

Conversation histories can be truncated, but that can lead to losing relevant information that appeared early on.

A better solution is to pass only the relevant conversational context on each subsequent call.

For this, RedisVL has the `SemanticSessionManager`, which uses vector similarity search to return only semantically relevant sections of the conversation.

In [ ]:
from redisvl.extensions.session_manager import SemanticSessionManager
semantic_session = SemanticSessionManager(name='student tutor')

semantic_session.add_messages(chat_session.get_recent(top_k=8))

In [ ]:
prompt = "what have I learned about the size of England?"
semantic_session.set_distance_threshold(0.35)
##context = semantic_session.get_relevant(prompt)
context = semantic_session.get_recent() ## testing GHA
for message in context:
    print(message)

You can adjust the degree of semantic similarity needed to be included in your context.

Setting a distance threshold close to 0.0 will require an exact semantic match, while a distance threshold of 1.0 will include everthing.

In [ ]:
semantic_session.set_distance_threshold(0.7)

##larger_context = semantic_session.get_relevant(prompt)
larger_context = semantic_session.get_recent() ##
for message in larger_context:
    print(message)

## Conversation control

LLMs can hallucinate on occasion and when this happens it can be useful to prune incorrect information from conversational histories so this incorrect information doesn't continue to be passed as context.

In [ ]:
semantic_session.store(prompt="what is the smallest country in Europe?",
                       response="Monaco is the smallest country in Europe at 0.78 square miles." # Incorrect. Vatican City is the smallest country in Europe
                       )

# get the key of the incorrect message
context = semantic_session.get_recent(top_k=1, raw=True)
bad_key = context[0]['entry_id']
semantic_session.drop(bad_key)

corrected_context = semantic_session.get_recent()
for message in corrected_context:
    print(message)

In [ ]:
chat_session.clear()